# GUI base demo of Stable Diffusion by [Junaid Afzal](https://github.com/imJunaidAfzal)
## You wil get Live url after running, click and enjoy sd-xl free.

### Note: If you get any dependancy error, restart runtime after running setup section.

# Setup

## Deps

In [ ]:
!pip install invisible_watermark transformers accelerate safetensors

In [ ]:
!pip install diffusers

In [ ]:
!pip install omegaconf

In [ ]:
!pip install gradio

## Project structure

In [ ]:
# Dont Change this
import os
if not os.path.exists("/content/images"):
    os.mkdir("/content/images")

!wget -O "/content/images/error.png" https://raw.githubusercontent.com/imJunaidAfzal/Prompt-Engineering/main/images/error.png


# SD-XL Pipeline code

In [ ]:
from diffusers import DiffusionPipeline
import torch
from diffusers import (
    DDIMScheduler,
    DPMSolverSinglestepScheduler,
    DDPMScheduler,
    EulerAncestralDiscreteScheduler,
    HeunDiscreteScheduler,
    KDPM2AncestralDiscreteScheduler,
    LMSDiscreteScheduler,
    UniPCMultistepScheduler,
    KDPM2DiscreteScheduler,
    DEISMultistepScheduler,
    DPMSolverSDEScheduler,
    PNDMScheduler,
    EulerDiscreteScheduler,
    DPMSolverMultistepScheduler
)

pipe = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16"
)
pipe.to("cuda")

In [ ]:
import gradio as gr
from PIL import Image
# Create a dictionary of scheduler names and their corresponding classes
scheduler_classes = {
    "DDIM Scheduler": DDIMScheduler,
    "DPMSolverSinglestep Scheduler": DPMSolverSinglestepScheduler,
    "DDPMScheduler": DDPMScheduler,
    "EulerAncestralDiscrete Scheduler": EulerAncestralDiscreteScheduler,
    "HeunDiscrete Scheduler": HeunDiscreteScheduler,
    "KDPM2AncestralDiscrete Scheduler": KDPM2AncestralDiscreteScheduler,
    "LMSDiscrete Scheduler": LMSDiscreteScheduler,
    "UniPCMultistep Scheduler": UniPCMultistepScheduler,
    "KDPM2Discrete Scheduler": KDPM2DiscreteScheduler,
    "DEISMultistep Scheduler": DEISMultistepScheduler,
    "DPMSolverSDE Scheduler": DPMSolverSDEScheduler,
    "PNDMScheduler": PNDMScheduler,
    "EulerDiscrete Scheduler": EulerDiscreteScheduler,
    "DPMSolverMultistep Scheduler": DPMSolverMultistepScheduler
}
def generate_image(prompt, negative_prompt, width, height, num_inference_steps, seed, guidance_scale, scheduler_name):
    try:
        scheduler_class = scheduler_classes[scheduler_name]
        generator = torch.Generator(device="cuda").manual_seed(seed)
        pipe.scheduler = scheduler_classes[scheduler_name].from_config(pipe.scheduler.config)
        image = pipe(
            prompt=prompt,
            negative_prompt=negative_prompt,
            width=width,
            height=height,
            num_inference_steps=num_inference_steps,
            generator=generator,
            guidance_scale=guidance_scale
        )
        # image.images[0]

        # Add your code here to generate the image using the selected scheduler class
        # You'll need to create an instance of the selected scheduler class and use it for image generation.

        return "OK", image.images[0]
    except Exception as exp:
        return str(exp), Image.open("/content/images/error.png")

with gr.Blocks() as ui:
    with gr.Row():
            gr.Markdown("<center><h1>SD-XL Demo</h1></center>")
    with gr.Row():

        with gr.Column():
            with gr.Group():
                with gr.Row():
                    prompt = gr.components.Textbox(label="Prompt")
                with gr.Row():
                    negative_Prompt = gr.components.Textbox(label="Negative Prompt")
                with gr.Row():
                    width = gr.components.Slider(label="Width", minimum=1, maximum=2024, step=1, value=1024, interactive=True)
                    height = gr.components.Slider(label="Height", minimum=1, maximum=2024, step=1, value=1024, interactive=True)
                with gr.Row():
                    inference_steps = gr.components.Slider(label="Number of Inference Steps", minimum=1, maximum=500, value=30, step=1, interactive=True)
                with gr.Row():
                    seed = gr.components.Slider(label="Seed", minimum=1, maximum=9999999999999, step=1, interactive=True)
                with gr.Row():
                    guidance_scale = gr.components.Slider(label="Guidance Scale", minimum=0, maximum=20, step=0.01, value=7.5, interactive=True)
                with gr.Row():
                    scheduler = gr.components.Dropdown(label="Select Scheduler", choices=list(scheduler_classes.keys()), value="EulerAncestralDiscrete Scheduler", interactive=True)
                with gr.Row():
                    gen_button = gr.Button("Generate Image")

        with gr.Column():
            image_output = gr.Image(interactive=False)
            status = gr.components.Textbox(label="Status", interactive=False)

        with gr.Column():
            gen_button.click(
                fn=generate_image,
                inputs=[prompt, negative_Prompt, width, height, inference_steps, seed, guidance_scale, scheduler],
                outputs=[status, image_output]
            )


ui.launch(share=True, debug=True)
